In [1]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr
import glob
import warnings

In [4]:
obs_gd = pd.read_csv('A_TEMP-20160101-20201231.csv',skiprows=[0,1,2,3,4,8])

In [5]:
#process the raw observations into a readable dataframe
obs_latlonID = obs_gd[0:2]
obs_gd_data = obs_gd[2:].reset_index(drop=True)
obs_gd_data.loc[:,['Station ID']] = pd.to_datetime(obs_gd_data['Station ID'])
obs_gd_data = obs_gd_data.replace(-99999,np.nan)
obs_gd_data = obs_gd_data.rename(columns={"Station ID": "Time"})

In [10]:
#filter data from June to September
obs_gd_data['year'] = obs_gd_data.Time.dt.year
obs_gd_data['month'] = obs_gd_data.Time.dt.month
obs_gd_data['day'] = obs_gd_data.Time.dt.day

obs_gd_data = obs_gd_data[(obs_gd_data['month']>=6)&(obs_gd_data['month']<=9)].reset_index(drop=True)

del obs_gd_data['year']
del obs_gd_data['month']
del obs_gd_data['day']

In [11]:
#calculate missing value percentage
obs_gd_data.loc['nan_num'] = obs_gd_data.isnull().sum(axis=0)
obs_gd_data.loc['nan_percentage'] = obs_gd_data.loc['nan_num']/14640
obs_gd_data = obs_gd_data.T

In [15]:
#pick out observations with missing value percentage less than 10%
obs_gd_data_10 = obs_gd_data.drop(obs_gd_data[obs_gd_data['nan_percentage']>0.1].index)
obs_gd_data_10 = obs_gd_data_10.T

In [17]:
obs_gd_data_10_cp = obs_gd_data_10.copy()
obs_gd_data_10_cp.drop(['nan_num','nan_percentage'],inplace=True)

In [19]:
#calculate mean of filtered observations
obs_gd_data_10_cp.loc['mean'] = obs_gd_data_10_cp.mean()
obs_gd_data_10_mean = obs_gd_data_10_cp.loc['mean']
obs_gd_data_10_mean = pd.DataFrame(obs_gd_data_10_mean)
obs_gd_data_10_mean.drop(index=['Time'],inplace=True)

In [23]:
#find out the lat and lon for each station
obs_latlonID = obs_latlonID.T
obs_latlonID.drop(['Station ID'],inplace=True)
obs_gd_data_10_mean = obs_gd_data_10_mean.reset_index()
obs_latlonID = obs_latlonID.reset_index()
obs_gd_data_10_mean = pd.merge(obs_gd_data_10_mean,obs_latlonID,on=['index'],how='left')
obs_gd_data_10_t2 = obs_gd_data_10_t2.rename(columns={0: "lat",1: "lon","index": "station_name"})

In [29]:
obs_gd_data_10_t2.to_csv('observation_mean.csv',index=False)